In [1]:
import pandas as pd
import numpy as np
#import re
#import xlrd

# fileName - iterate through entire folder :)
fileName = 'Original_FiveAssetClasses.xlsx'
#fileNameOtherDevices = 'Other Device Numbers.xls'

# Read xlsx file into dataframes
with pd.ExcelFile(fileName) as xlsx:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfTransformersV1 = pd.read_excel(xlsx, 'Transformers') # 280 rows
    dfSwitchesV1 = pd.read_excel(xlsx, 'Switches') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
    dfPolesV1 = pd.read_excel(xlsx, 'Poles') # 239 rows; 'Spot Number\n' col contains unique tx ids
    dfCablesV1 = pd.read_excel(xlsx, 'UGPrimaryCables')
    dfFusesV1 = pd.read_excel(xlsx, 'Fuses') # 44 items
    dfUGStructuresV1 = pd.read_excel(xlsx,'UGStructures')


Summary = {'Transformers:': dfTransformersV1.shape, 'Switches:': dfSwitchesV1.shape,'Poles:': dfPolesV1.shape, 
           'Cables:': dfCablesV1.shape, 'Fuses:':dfFusesV1.shape, 'UGStructures:':dfUGStructuresV1.shape}
dfSummary = pd.DataFrame(Summary)

# Make one copy
dfTransformersV2 = dfTransformersV1
dfSwitchesV2 = dfSwitchesV1
dfPolesV2 = dfPolesV1
dfCablesV2 = dfCablesV1
dfFusesV2 = dfFusesV1
dfUGStructuresV2 = dfUGStructuresV1

# 17 columns dropped
dropCommonColumns = ['OBJECTID','WORKORDERID','FIELDVERIFY','COMMENTS','CREATIONUSER','DATECREATED','LASTUSER',
                     'DATEMODIFIED','WORKREQUESTID','DESIGNID','WORKLOCATIONID','WMSID','WORKFLOWSTATUS',
                     'WORKFUNCTION','GISONUMBER','GISOTYPENBR','OWNERSHIP']

def drop_columns(dfAssetClass, dropColumns):
    dfAssetClass = dfAssetClass.drop(dropColumns, axis=1)
    return dfAssetClass

#Drop all common columns 
dfSwitchesV2 = drop_columns(dfSwitchesV2, dropCommonColumns)
dfTransformersV2 = drop_columns(dfTransformersV2, dropCommonColumns)
dfFusesV2 = drop_columns(dfFusesV2,dropCommonColumns)
dfCablesV2 = drop_columns(dfCablesV2, dropCommonColumns)
dfUGStructuresV2 = drop_columns(dfUGStructuresV2, dropCommonColumns)
dfPolesV2 = drop_columns(dfPolesV2, dropCommonColumns)

# Make one copy
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

SummaryV2 = {'Transformers:': dfTransformers.shape, 'Switches:': dfSwitches.shape,'Poles:': dfPoles.shape, 
           'Cables:': dfCables.shape, 'Fuses:':dfFuses.shape, 'UGStructures:':dfUGStructures.shape}
dfSummaryV2 = pd.DataFrame(SummaryV2)
print(dfSummary)
print(dfSummaryV2)
# print(dfSummary-dfSummaryV2) # shows 17 cols dropped

   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       39      46      31         52             57             40
   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       22      29      14         35             40             23


In [2]:
# Save future wait times while running
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

# Switches
fileNameOtherDevices = 'Other Device Numbers.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameOtherDevices) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfSwitchGears = pd.read_excel(xls, 'SWITCHGEARS') # 280 rows

dropSGcols = ['Switch Gear', 'Adrs #','Location','City','Notes','To Type','Inst. Date','Mftr.','Catalog#','Serial#',
             'DOM','Comments']

dfSwitchGears = drop_columns(dfSwitchGears, dropSGcols)
#dfSwitchGears = dfSwitchGears.dropna() # drop all rows with NaN values

def new_columns(dfAssetClass, numAssetRows, columnID):
    dfAssetClass[columnID] = pd.DataFrame(np.empty([numAssetRows,1]).cumsum(axis=1))
    dfAssetClass.loc[:,columnID] = np.nan
    return dfAssetClass[columnID]

dictSGassetSubclass = {'PMH-3':'Air-Insulated Live Front','PMH-5':'Air-Insulated Live Front',
                       'PMH-9':'Air-Insulated Live Front','PMH-11':'Air-Insulated Live Front',
                       'PME-9':'Air-Insulated Dead Front','PME-10':'Air-Insulated Dead Front',
                       'PME-11':'Air-Insulated Live Front','VISTA-321':'SF6-Insulated Switch',
                       'VISTA-422':'SF6-Insulated Switch','VISTA-431':'SF6-Insulated Switch',
                       '422':'S&C Elec','431':'S&C Elec','321':'S&C Elec','G&W':'G&W',
                       'NET':'Carte Elec Ltd'}
# 'Type' -> 'PMH'
# 'Loc_No' -> '149-S'
dfSwitchGears['Type'] = dfSwitchGears['Type'].fillna(method='ffill')
numSGrows = len(dfSwitchGears['Loc_No'])
dfSwitchGears['Subclass'] = new_columns(dfSwitchGears, numSGrows, 'Subclass')
dfSwitchGears =dfSwitchGears.astype(str)
#dfSwitchGears['Loc_No'] = dfSwitchGears.iloc[:,'Loc_No'].apply[s.lstrip("0") for s in listOfNum]
dfSwitchGears['Loc_No'] = [s.lstrip("0") for s in dfSwitchGears['Loc_No']]
dfSwitchGears['Subclass'] = dfSwitchGears['Type'].apply(lambda x: dictSGassetSubclass[x])

# MasterFile = pd.ExcelWriter('V2_GIS_SGs.xlsx')
# dfSwitchGears.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()
# print(dfSwitchGears.head(4))

In [3]:
dropSwitchesCols = ['ANCILLARYROLE','ENABLED','FEEDERINFO','ELECTRICTRACEWEIGHT','LOCATIONID','GPSDATE','LABELTEXT',
                    'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'MAXOPERATINGVOLTAGE','MAXCONTINUOUSCURRENT','PRESENTPOSITION_R', 
                    'PRESENTPOSITION_Y', 'PRESENTPOSITION_B','NORMALPOSITION_R','NORMALPOSITION_Y','NORMALPOSITION_B', 
                    'SCADACONTROLID', 'SCADAMONITORID','PREFERREDCIRCUITSOURCE','TIESWITCHINDICATOR',
                    'GANGOPERATED', 'MANUALLYOPERATED','FEATURE_STATUS','HYPERLINK','HYPERLINK_PGDB','SYMBOLROTATION',
                    'INSULATOR_MATERIAL']

# drop columns and drop rows that are not UG Switch
dfSwitches = drop_columns(dfSwitches,dropSwitchesCols)
dfSwitches = dfSwitches[dfSwitches.SUBTYPECD == 6]
# df.query('line_race != 0')
# df = df[df.line_race != 0]

# Rename Switch columns
dfSwitches = dfSwitches.rename(columns={'SUBTYPECD':'Asset Class','DEVICENUMBER':'ID','COMPATIBLEUNITID':'Asset Subclass',
                                        'PHASEDESIGNATION':'PHASING','FEEDERID':'CIRCUIT', 'FEEDERID2':'TIE_FEEDER',
                                        'INSTALLATIONDATE':'INSTALL_YEAR'})
# Separate year
dfSwitches['INSTALL_YEAR'] = dfSwitches['INSTALL_YEAR'].apply(lambda x: x.year)

# Add additional columns and fill with NaNs
numSwitchRows = len(dfSwitches['ID'])
dfSwitches['HI'] = new_columns(dfSwitches,numSwitchRows, 'HI')
dfSwitches['TX_PHASE'] = new_columns(dfSwitches,numSwitchRows, 'TX_PHASE')
dfSwitches['IN_VALLEY'] = new_columns(dfSwitches,numSwitchRows, 'IN_VALLEY')
dfSwitches['PRID'] = new_columns(dfSwitches,numSwitchRows, 'PRID')

print(dfSwitches.shape) # (537, 11) with all switches, (260,11) with subtype=6 ('Switch Switchgear')
print('dfSG Shape:', dfSwitchGears.shape) #(111, 3)

(260, 11)
dfSG Shape: (451, 3)


In [4]:
# Replace 'Asset Subclass' col with actual names
dfSwitches['ID'] = dfSwitches['ID'].astype(str)
dfSwitches=pd.merge(dfSwitches, dfSwitchGears, how='left', left_on='ID', right_on='Loc_No')
#dfSwitches['ID'] = dfSwitchGears['Loc_No'].apply(lambda x: )
#df.merge(df1, on='sku', how='left')
# print(len(pd.unique(dfSwitchGears['Loc_No'].values.ravel()))) # 111

switchesDropMoreCols = ['Asset Subclass', 'Loc_No']
dfSwitches = dfSwitches.drop(switchesDropMoreCols, axis=1)
dfSwitches = dfSwitches.rename(columns={'Subclass':'Asset Subclass'})
#Rearrange columns
dfSwitches=dfSwitches[['Asset Class', 'Asset Subclass', 'ID','CIRCUIT','INSTALL_YEAR','TIE_FEEDER','PHASING','IN_VALLEY','HI','TX_PHASE','PRID','Type']]

MasterFile = pd.ExcelWriter('V2_GIS_Switches.xlsx')
dfSwitches.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

In [ ]:
print(dfSwitches.shape)
print('dfSG Shape:', dfSwitchGears.shape)
print(dfSwitchGears.uniquevalues())
# print(dfSwitches.head(5))
# print(dfSwitchGears.head(10))

In [60]:
# colNames = {'Transformers:': list(dfTransformers.columns), 'Switches:': list(dfSwitches.columns),
#            'Poles:': list(dfPoles.columns), 'Cables:': list(dfCables.columns), 'Fuses:':list(dfFuses.columns),
#            'UGStructures:':list(dfUGStructures.columns)}
# dfColNames = pd.Series(colNames)
# print(dfColNames['Transformers:'])

# Cables
dropCablesCols = ['ENABLED', 'INSTALLATIONDATE', 'FEEDERID', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 
                 'LENGTHSOURCE', 'MEASUREDLENGTH', 'LENGTHUOMCODE', 'WIRECOUNT', 'SUBTYPECD', 'LABELTEXT', 
                 'COMPATIBLEUNITID', 'PHASEDESIGNATION', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'ISFEEDERTRUNK', 
                 'NEUTRALUSECD', 'FEATURE_STATUS', 'CONDUCTOR_REJUVENATION', 'SHAPE_Length']



#print(dfSwitches.head(5))


# Rename OH Tx columns

# Rename UG Tx columns

# Rename Distribution Poles columns

# Rename UG Cable columns



  INSTALL_YEAR CIRCUIT TIE_FEEDER  Asset Class  Asset Subclass      ID
0   1981-09-01   3S2-1        NaN            6            1233    92-S
1   1900-01-01   5S1-3        NaN            6            1233   336-S
2   2008-06-01   3S2-1        NaN            6            1233   360-S
3   2015-06-12   5S2-3        NaN            6            1233  380-MS
4   1987-10-01   5S2-3      6S2-1            3             193    6-MS


In [61]:
# Poles
dropPolesCols = ['INSTALLATIONDATE', 'SYMBOLROTATION', 'GPSDATE', 'SUBTYPECD', 'LABELTEXT', 'COMPATIBLEUNITID', 
                 'STRUCTURENUMBER', 'FEATURE_STATUS', 'STREETLIGHT_FACILITY', 'REPLACED_DATE_MM_DD_YYYY', 
                 'DEVICENUMBER', 'CONDITION', 'CONDITION_STATUS', 'CONDITION_DATE']



  INSTALL_YEAR CIRCUIT TIE_FEEDER  Asset Class  Asset Subclass      ID  HI  \
0   1981-09-01   3S2-1        NaN            6            1233    92-S NaN   
1   1900-01-01   5S1-3        NaN            6            1233   336-S NaN   
2   2008-06-01   3S2-1        NaN            6            1233   360-S NaN   
3   2015-06-12   5S2-3        NaN            6            1233  380-MS NaN   
4   1987-10-01   5S2-3      6S2-1            3             193    6-MS NaN   

   TX_PHASE IN_VALLEY  
0       NaN        No  
1       NaN        No  
2       NaN        No  
3       NaN        No  
4       NaN        No  


In [66]:
#Transformer

# separate into UG and OH

dropTransformersCols = ['ANCILLARYROLE', 'ENABLED', 'INSTALLATIONDATE', 'FEEDERID', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 'SYMBOLROTATION', 'GPSDATE', 'SUBTYPECD', 'LABELTEXT', 'COMPATIBLEUNITID', 'PHASEDESIGNATION', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'GROUNDREACTANCE', 'GROUNDRESISTANCE', 'MAGNETIZINGREACTANCE', 'MAGNETIZINGRESISTANCE', 'HIGHSIDEGROUNDREACTANCE', 'HIGHSIDEGROUNDRESISTANCE', 'HIGHSIDEPROTECTION', 'LOCATIONTYPE', 'DEVICENUMBER', 'FAULTINDICATOR', 'COOLINGTYPE', 'FEATURE_STATUS', 'KVA', 'UNITS', 'DEMAND_KVA', 'DEMAND_DATE_MM_DD_YYYY', 'STREET_LIGHT_FACILITY', 'HIGHSIDECONFIGURATION', 'LOWSIDECONFIGURATION', 'LOWSIDEGROUNDRESISTANCE', 'LOWSIDEVOLTAGE', 'LATITUDE', 'LONGITUDE', 'RATEDKVA']


In [67]:
# Fuses

dropFusesCols = ['ANCILLARYROLE', 'ENABLED', 'INSTALLATIONDATE', 'FEEDERID', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 'GPSDATE', 'SUBTYPECD', 'LABELTEXT', 'COMPATIBLEUNITID', 'PHASEDESIGNATION', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'MAXCONTINUOUSCURRENT', 'MAXINTERRUPTINGCURRENT', 'MAXOPERATINGVOLTAGE', 'PRESENTPOSITION_R', 'PRESENTPOSITION_Y', 'PRESENTPOSITION_B', 'NORMALPOSITION_R', 'NORMALPOSITION_Y', 'NORMALPOSITION_B', 'DEVICENUMBER', 'FUSELINKSIZE', 'FEATURE_STATUS', 'SYMBOLROTATION', 'INSULATOR_MATERIAL']


   INSTALL_YEAR CIRCUIT TIE_FEEDER  Asset Class  Asset Subclass      ID  HI  \
0          1981   3S2-1        NaN            6            1233    92-S NaN   
1          1900   5S1-3        NaN            6            1233   336-S NaN   
2          2008   3S2-1        NaN            6            1233   360-S NaN   
3          2015   5S2-3        NaN            6            1233  380-MS NaN   

   TX_PHASE IN_VALLEY  
0       NaN        No  
1       NaN        No  
2       NaN        No  
3       NaN        No  


In [ ]:
# Drop_Switch_Columns = ['OBJECTID', 'ANCILLARYROLE', 'ENABLED', 'WORKORDERID', 'FIELDVERIFY', 'COMMENTS','CREATIONUSER', 
#                        'DATECREATED', 'LASTUSER', 'DATEMODIFIED', 'WORKREQUESTID', 'DESIGNID','WORKLOCATIONID', 'WMSID', 
#                        'WORKFLOWSTATUS', 'WORKFUNCTION', 'FEEDERINFO','ELECTRICTRACEWEIGHT', 'LOCATIONID', 'GPSDATE', 
#                        'GISONUMBER', 'GISOTYPENBR', 'LABELTEXT','OWNERSHIP', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE',
#                        'MAXOPERATINGVOLTAGE', 'MAXCONTINUOUSCURRENT', 'PRESENTPOSITION_R', 'PRESENTPOSITION_Y', 
#                        'PRESENTPOSITION_B', 'NORMALPOSITION_R', 'NORMALPOSITION_Y', 'NORMALPOSITION_B', 'SCADACONTROLID', 
#                        'SCADAMONITORID', 'PREFERREDCIRCUITSOURCE', 'TIESWITCHINDICATOR', 'GANGOPERATED', 'MANUALLYOPERATED',
#                         'FEATURE_STATUS', 'HYPERLINK', 'HYPERLINK_PGDB', 'SYMBOLROTATION', 'INSULATOR_MATERIAL']


# %timeit
# Strip '\n' from column headers
dfTopology.rename(columns=lambda x: x.replace('\n',''), inplace=True)
dfSpotLoads.rename(columns=lambda x: x.replace('\n',''), inplace=True)
dfLoads.rename(columns=lambda x: x.replace('\n',''), inplace=True)
dfCables.rename(columns=lambda x: x.replace('\n',''), inplace=True)
dfSwitches.rename(columns=lambda x: x.replace('\n',''), inplace=True)
dfNodes.rename(columns=lambda x: x.replace('\n',''), inplace=True)
dfOHlines.rename(columns=lambda x: x.replace('\n',''), inplace=True)
dfFuses.rename(columns=lambda x: x.replace('\n',''), inplace=True)

# Rename column headers
dfTopology.rename(columns=lambda x: 'Topology_'+x, inplace=True)
#dfSpotLoads.rename(columns=lambda x: 'SpotLoads_'+x, inplace=True)
dfLoads.rename(columns=lambda x: 'Loads_'+x, inplace=True)
dfCables.rename(columns=lambda x: 'Cables_'+x, inplace=True)
dfSwitches.rename(columns=lambda x: 'Switches_'+x, inplace=True)
dfNodes.rename(columns=lambda x: 'Nodes_'+x, inplace=True)
dfOHlines.rename(columns=lambda x: 'OHlines_'+x, inplace=True)
dfFuses.rename(columns=lambda x: 'Fuses_'+x, inplace=True)

# Merge assets: switch, transformers, fuses, cables, OHlines to Node worksheet
dfNodesMaster = pd.merge(dfNodes, dfSwitches, how='outer', left_on='Nodes_Node Id', right_on ='Switches_From Node')
dfNodesMaster = pd.merge(dfNodesMaster, dfLoads, how='outer', left_on='Nodes_Node Id', right_on='Loads_From Node')
dfNodesMaster = pd.merge(dfNodesMaster, dfFuses, how='outer', left_on='Nodes_Node Id', right_on='Fuses_From Node')
#dfNodesMaster = pd.merge(dfNodesMaster, dfOHlines, how='outer', left_on='Nodes_Node Id', right_on='OHlines_From Node')
dfNodesMaster = pd.merge(dfNodesMaster, dfCables, how='outer', left_on='Nodes_Node Id', right_on='Cables_From Node')
# print(dfNodesMaster.head(3))
print(len(dfNodesMaster.columns))

dfNodesMaster = dfNodesMaster.rename(columns={'Loads_Total CkVA(kVA)':'Nameplate', 'Loads_Spot Number':'TransformerID'})
dfNodesCopy = dfNodesMaster
#print(dfNodesCopy.dtypes)

#Change to str
dfNodesCopy['Cables_From Node']= dfNodesCopy['Cables_From Node'].astype(str)
dfNodesCopy['Cables_To Node']= dfNodesCopy['Cables_To Node'].astype(str)

In [5]:
# Split 'Nodes_Node Id' to 'NodeID_1' and 'NodeID_2' for 'SwitchRegion'
dfNodesCopy['NodeID_1'], dfNodesCopy['NodeID_2'] = zip(*dfNodesCopy['Nodes_Node Id'].
                                                       apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))

#******************************#
#***DIFFERET FROM V5 BEGINS****#
#******************************#
dfNodesCopy['Cables_FromNodeID_1'], dfNodesCopy['Cables_FromNodeID_2'] = zip(*dfNodesCopy['Cables_From Node'].
                                                       apply(lambda x: x.split('_') if '_' in x else (x,np.nan)))
dfNodesCopy['Cables_ToNodeID_1'], dfNodesCopy['Cables_ToNodeID_2'] = zip(*dfNodesCopy['Cables_To Node'].
                                                        apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
                # Columns 'Cables_FromNodeID_2' and 'Cables_ToNodeID_2' dropped in Cables_drop_cols
#******************************#
#***DIFFERET FROM V5 ENDS******#
#******************************#

In [6]:
#Switch region col a.fill(numpy.nan), a[:] = numpy.nan
Num_rows = len(dfNodesCopy['Nodes_Network Id'])
dfNodesCopy['SwitchRegion'] = pd.DataFrame(np.empty([Num_rows,1]).cumsum(axis=1))
dfNodesCopy['CablesSwitchRegionFrom'] = pd.DataFrame(np.empty([Num_rows,1]).cumsum(axis=1))
dfNodesCopy['CablesSwitchRegionEnd'] = pd.DataFrame(np.empty([Num_rows,1]).cumsum(axis=1))
# avoid chain indexing - http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
dfNodesCopy.loc[:,'SwitchRegion'] = np.nan
dfNodesCopy.loc[:,'CablesSwitchRegionFrom'] = np.nan
dfNodesCopy.loc[:,'CablesSwitchRegionEnd'] = np.nan

# V6 changes
#df['Normalized'] = np.where(df['Currency'] == '$', df['Budget'] * 0.78125, df['Budget'])
#'Cables_FromNodeID_2' and 'Cables_ToNodeID_2'
dfNodesCopy['CablesSwitchRegionFrom'] = dfNodesCopy['Cables_FromNodeID_1'].apply(lambda x: x if '-' in x else np.nan)
dfNodesCopy['CablesSwitchRegionEnd'] = dfNodesCopy['Cables_ToNodeID_1'].apply(lambda x: x if '-' in x else np.nan)
dfNodesCopy['SwitchRegion'] = dfNodesCopy['NodeID_1'].apply(lambda x: x if '-' in x else np.nan)

#FillNA
dfNodesCopy['SwitchRegion'] = dfNodesCopy['SwitchRegion'].fillna(method='ffill')
dfNodesCopy['CablesSwitchRegionFrom'] = dfNodesCopy['CablesSwitchRegionFrom'].fillna(method='ffill')
dfNodesCopy['CablesSwitchRegionEnd'] = dfNodesCopy['CablesSwitchRegionEnd'].fillna(method='ffill')

#V5
#dfNodesCopy['SwitchRegion'] = dfNodesCopy['NodeID_1'].apply(lambda x: x if '-' in x else np.nan)

#http://stackoverflow.com/questions/27905295/how-to-replace-nans-by-preceding-values-in-pandas-dataframe
# df.fillna(method='ffill')
# http://stackoverflow.com/questions/11497206/regular-expression-for-letters-dash-underscore-numbers-and-space

# Remove columns - temporary list for now
Nodes_drop_cols = ['Nodes_Phase','Nodes_Node Id', 'NodeID_1','NodeID_2'] #['Nodes_Network Id','Nodes_Node Id','Nodes_Phase'] 
Switches_drop_cols = ['Switches_Network Id','Switches_Equipment Id','Switches_Device Type','Switches_Status',
                         'Switches_Phase','Switches_From Node','Switches_Voltage(kV)'] 
                        #'Switches_Section Id','Switches_State','Switches_Rating(A)'
Tx_drop_cols = ['Loads_Network Id','Loads_Section Id','Loads_Status','Loads_From Node','Loads_Spot Type',
                   'Loads_Dist Number','Loads_Dist Type','Loads_Total kVA(kVA)','Loads_Total kW(kW)','Loads_Total kvar',
                   'Loads_Aver. PF(%)','Loads_Total kWh(kWh)','Loads_Total Cust','Loads_Phase Type','Loads_Config',
                   'Loads_Locked','Loads_Load Model'] #'Loads_TransformerID','Loads_Phase','Loads_Nameplate',

Fuses_drop_cols =['Fuses_Network Id', 'Fuses_Status','Fuses_State','Fuses_Phase','Fuses_Manufacturer', 
                    'Fuses_Model', 'Fuses_Voltage(kV)', 'Fuses_Voltage Class', 'Fuses_Standard', 'Fuses_Rating(A)',
                  'Fuses_Rating','Fuses_Interrupting Rating(A)', 'Fuses_From Node', 'Fuses_To Node'] 
                    #  'Fuses_Section Id', 'Fuses_Equipment Id', Fuses_Rating' 

#OHlines_deleted_cols =['OHlines_Network Id','OHlines_Phase','OHlines_Cond R','OHlines_Cond Y',
                       #'OHlines_Cond B','OHlines_Neutral 1','OHlines_Neutral 2','OHlines_Spacing']
                        #'OHlines_Section Id','OHlines_Length(m)',
OHlines_drop_cols=[]

#V6 changes
Cables_drop_cols =['Cables_Network Id','Cables_Equipment Id','Cables_Line Id','Cables_Status','Cables_Phase',
                      'Cables_# parallel','Cables_Manufacturer','Cables_Standard',
                      'Cables_Rated Voltage(kV)','Cables_Ampacity(A)','Cables_Withstand(A)','Cables_Cable Type',
                      'Cables_Conductor Material','Cables_Sheathed','Cables_Concentric Neutrals','Cables_Line R1(ohms)',
                      'Cables_Line X1(ohms)','Cables_Line B1(µS)','Cables_Line R0(ohms)','Cables_Line X0(ohms)',
                      'Cables_Line B0(µS)','Cables_Harmonic Model', 'Cables_FromNodeID_1','Cables_ToNodeID_1',
                      'Cables_FromNodeID_2','Cables_ToNodeID_2','Cables_From Node', 'Cables_To Node',] 
                    #'Cables_From Node','Cables_To Node','Cables_Length(m)','Cables_Size','Cables_insulation'

# Poles_deleted_cols=[]

combined_drop_cols = (Nodes_drop_cols + Switches_drop_cols + Tx_drop_cols + Fuses_drop_cols + 
                      OHlines_drop_cols + Cables_drop_cols)

print('Number of columns deleted: ', len(combined_drop_cols))
print('Number of rows: ', len(dfNodesCopy['Nodes_Network Id']))

#Drop columns
dfNodesCopy=dfNodesCopy.drop(combined_drop_cols, axis=1)

#print('new cols: ', len(dfNodesCopy.columns))
print('Number of remaining columnns: ', len(dfNodesCopy.columns))
#print(dfNodesCopy.head(7))

Number of columns deleted:  70
Number of rows:  735
Number of remaining columnns:  16


In [7]:
MasterFile = pd.ExcelWriter('V7_NodeIDs.xlsx')
dfNodesCopy.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

In [26]:
#SwitchGrouped = dfNodesCopy.groupby('SwitchRegion')
#print(SwitchGrouped.head(3))

In [41]:
# #  Ctrl + A
# # Ctrl + / to uncomment

# # ****************************
# # A. NODES sheet
# # ****************************
# # 1. Split 'Node Id' to 'NodeID_1' and 'NodeID_2'
# dfNodes['NodeID_1'], dfNodes['NodeID_2'] = zip(*dfNodes['Node Id'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Create a 'Copy' dataframe
# dfNodesCopy = pd.DataFrame(dfNodes)
# # 3. Rename all column headers to 'Nodes_' + x
# dfNodesCopy.rename(columns=lambda x: 'Nodes_'+x, inplace=True)
# #print(dfNodes_Copy.count())

# # ****************************
# # B. MASTER SPREADSHEET
# # ****************************
# # Copy dfNodesCopy into dfMaster
# dfMaster = pd.DataFrame(dfNodesCopy)
# # print(dfMaster.count())
# # Nodes_NodeID_1 and Nodes_NodeID_2 are keys

# # ****************************
# # C. Topology sheet
# # ****************************
# # 1. No renaming here,so freate a 'copy' dataframe
# dfTopologyCopy = pd.DataFrame(dfTopology)
# # 2. Rename all column headers to 'Topology_' + x
# dfTopologyCopy.rename(columns=lambda x: 'Topology_'+x, inplace=True)
# #print(dfTopologyCopy.count())

# # 3. Combine topology sheet
# # pd.merge(frame_1, frame_2, left_on = 'county_ID', right_on = 'countyid')
# # dfFinal = 
# # Topology - more match with 'Topology_Coord. Y' over 'Topology_Coord. X'
# dfMaster = pd.merge(dfMaster, dfTopologyCopy, how='outer', left_on='Nodes_NodeID_2', right_on ='Topology_Coord. Y')
# #print(dfMaster.count())


# # ****************************
# # D. Fuses sheet 
# # ****************************
# # 1. Split 'From Node' to 'FromNode_xCoord' and 'FromNode_yCoord'
# dfFuses['FromNode_xCoord'], dfFuses['FromNode_yCoord'] = zip(*dfFuses['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Split 'To Node' to 'ToNode_FuseID' and 'ToNode_FdrID'
# dfFuses['ToNode_FuseID'], dfFuses['ToNode_FdrID'] = zip(*dfFuses['To Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 3. Create a 'Copy' dataframe
# dfFusesCopy = pd.DataFrame(dfFuses)
# # 4. Rename all column headers to 'Fuses_' + x
# dfFusesCopy.rename(columns=lambda x:'Fuses_'+x, inplace=True)
# # 5. Combine Fuses sheet with Master
# dfMaster = pd.merge(dfMaster, dfFusesCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Fuses_FromNode_xCoord')
# #print(dfMaster.count())

# # ****************************
# # D1. Output excel file - For VERIFICATION purposes
# # ****************************
# # Verify the excel file 
# # http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.to_excel.html
# # http://stackoverflow.com/questions/29974672/writing-pandas-dataframe-to-excel-with-different-formats-for-different-columns
# # MasterFile = pd.ExcelWriter('master.xlsx')
# # dfMaster.to_excel(MasterFile, 'Sheet1')
# # MasterFile.save()


In [37]:
# Ctrl + / to uncomment

# # ****************************
# # E. Switch sheet 
# # ****************************
# # 1. Split 'From Node' to 'FromNode_1' and 'FromNode_2'
# dfSwitches['FromNode_1'], dfSwitches['FromNode_2'] = zip(*dfSwitches['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Create a 'Copy' dataframe
# dfSwitchesCopy = pd.DataFrame(dfSwitches)
# # 3. Rename all column headers to 'Switches_' + x
# dfSwitchesCopy.rename(columns=lambda x:'Switches_'+x, inplace=True)
# # 4. Combine Switches sheet with Master: 
# # 4.1 First with 'Switches_FromNode_1' - NodeID_1 also has '109-D'/'7-S' switch id :)
# dfMaster = pd.merge(dfMaster, dfSwitchesCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Switches_FromNode_1')
# # 4.2 Second with 'Section Id' of FusesCopy - maybe not necessary

# # ****************************
# # F. Transformer aka "Loads" in CYME
# # ****************************
# # 
# # 1. Split 'From Node' to 'FromNode_1' and 'FromNode_2'
# dfLoads['FromNode_1'], dfLoads['FromNode_2'] = zip(*dfLoads['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Create a 'Copy' dataframe
# dfLoadsCopy = pd.DataFrame(dfLoads)
# # 3. Rename all column headers to Loads_' + x
# dfLoadsCopy.rename(columns=lambda x:'Loads_'+x, inplace=True)
# # 4. Combine all Loads with 'FromNode_1'  with dfMaster
# dfMaster = pd.merge(dfMaster, dfLoadsCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Loads_FromNode_1')
# # 4.2 May need to combine dfLoadsCopy with dfSpotLoads if tx nameplate rating not same


# #Plot
# %matplotlib inline
# import matplotlib.pyplot as plt
# pd.options.display.mpl_style = 'default'
# #dfSwitches.boxplot()
# #dfFusesCopy.boxplot(column="Fuses_Rating(A)")
# #mydf['CigarNum'] = mydf['CigarNum'].convert_objects(convert_numeric=True)
# dfFusesCopy['Fuses_FromNode_xCoord'] = dfFusesCopy['Fuses_FromNode_xCord'].convert_objects(convert_numeric=True)
# dfFusesCopy['Fuses_FromNode_yCoord'] = dfFusesCopy['Fuses_FromNode_yCord'].convert_objects(convert_numeric=True)
# #dfFusesCopy.plot(kind='scatter', x='Fuses_FromNode_xCoord', y='Fuses_FromNode_yCoord')


# # ****************************
# # D1. Output excel file
# # ****************************
# # Verify the excel file 
# # http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.to_excel.html
# # http://stackoverflow.com/questions/29974672/writing-pandas-dataframe-to-excel-with-different-formats-for-different-columns
# #MasterFile = pd.ExcelWriter('master.xlsx')
# #dfMaster.to_excel(MasterFile, 'Sheet1')
# #MasterFile.save()


In [8]:
# ****************************
# G. PRID to each region
# ****************************
# combine PRID to Tx?

# ****************************
# H. Cable 
# ****************************
# combine cable and conductors

# ****************************
# I. Conductors  
# ****************************
# combine poles

# ****************************
# J. Poles 
# ****************************
# Output excel file

